In [ ]:
import logging
from binance import Client, exceptions
import pandas as pd
import time
from datetime import datetime
import os
from dotenv import load_dotenv
from prometheus_client import start_http_server, Gauge, Counter, Histogram, Summary
from requests.exceptions import ConnectionError, Timeout

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Métricas Prometheus
# (Defina todas as suas métricas aqui)

# Iniciar o servidor HTTP do Prometheus em uma thread separada
start_http_server(8000)

# Configure o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Obtenha as chaves da API de variáveis de ambiente
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_API_SECRET')

if not api_key or not api_secret:
    logger.error("API Key ou API Secret não encontrada. Verifique o arquivo .env.")
    exit(1)

client = Client(api_key, api_secret, requests_params={'timeout': 20})

symbol_btc = "BTCUSDT"
symbol_eth = "ETHUSDT"
quantity_btc = 0.0011
quantity_eth = 0.01
interval = '1h'
setup = "9.1"

def calculate_percentage(current_price, target_price):
    return (target_price - current_price) / current_price * 100

def calculate_standard_deviation(prices):
    return pd.Series(prices).std()

def calculate_profit_factor(total_profit, total_loss):
    return total_profit / abs(total_loss) if total_loss != 0 else float('inf')

def safe_float_conversion(value):
    try:
        return float(value)
    except ValueError:
        return None

def read_trade_history(symbol):
    file_path = f'data/trade_history_{symbol}.csv'
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        if not df.empty:
            return df
    return pd.DataFrame()

def update_trade_history(df, sell_price, symbol):
    global total_profit, total_loss, total_trade_duration, successful_trades, total_trades, total_trade_volume, buy_prices, sell_prices
    df.at[df.index[-1], 'valor_venda'] = sell_price
    outcome = calculate_percentage(df.loc[df.index[-1], 'valor_compra'], sell_price)
    df.at[df.index[-1], 'outcome'] = outcome
    file_path = f'data/trade_history_{symbol}.csv'
    df.to_csv(file_path, index=False)
    transaction_outcome_metric.labels(symbol).observe(outcome)

    if outcome > 0:
        total_profit += outcome
        successful_trades += 1
    else:
        total_loss += outcome

    total_trades += 1
    total_trade_volume += df['quantidade_moeda'].iloc[-1]
    sell_prices.append(sell_price)
    total_profit_metric.labels(symbol).set(total_profit)
    total_loss_metric.labels(symbol).set(total_loss)
    success_rate = (successful_trades / total_trades) * 100 if total_trades > 0 else 0
    success_rate_metric.labels(symbol).set(success_rate)
    total_trades_metric.labels(symbol).inc()
    average_trade_duration = total_trade_duration / total_trades if total_trades > 0 else 0
    average_trade_duration_metric.labels(symbol).set(average_trade_duration)
    trade_volume_metric.labels(symbol).set(total_trade_volume)
    average_sell_price_metric.labels(symbol).set(sum(sell_prices) / len(sell_prices))
    profit_factor_metric.labels(symbol).set(calculate_profit_factor(total_profit, total_loss))

def update_metrics_on_buy(current_price, stoploss, stopgain, potential_loss, potential_gain, symbol):
    current_stoploss_metric.labels(symbol).set(stoploss)
    current_stopgain_metric.labels(symbol).set(stopgain)
    last_buy_price_metric.labels(symbol).set(current_price)
    buy_attempts_metric.labels(symbol).inc()
    successful_buys_metric.labels(symbol).inc()
    buy_price_spread_metric.labels(symbol).set(max(buy_prices) - min(buy_prices) if buy_prices else 0)
    potential_loss_metric.labels(symbol).set(potential_loss)
    potential_gain_metric.labels(symbol).set(potential_gain)

def update_metrics_on_sell(ticker, symbol):
    last_sell_price_metric.labels(symbol).set(ticker)
    successful_sells_metric.labels(symbol).inc()
    sell_price_spread_metric.labels(symbol).set(max(sell_prices) - min(sell_prices) if sell_prices else 0)

def get_current_balance(asset):
    try:
        balance_info = client.get_asset_balance(asset=asset)
        return float(balance_info['free'])
    except exceptions.BinanceAPIException as e:
        logger.error(f"Erro na API Binance ao obter saldo: {e}")
        return 0.0
    except Exception as e:
        logger.error(f"Erro inesperado ao obter saldo: {e}")
        return 0.0

def get_lot_size(symbol):
    try:
        info = client.get_symbol_info(symbol)
        for f in info['filters']:
            if f['filterType'] == 'LOT_SIZE':
                return float(f['stepSize'])
        return None
    except exceptions.BinanceAPIException as e:
        logger.error(f"Erro na API Binance ao obter LOT_SIZE: {e}")
        return None
    except Exception as e:
        logger.error(f"Erro inesperado ao obter LOT_SIZE: {e}")
        return None

def check_last_transaction(symbol):
    try:
        # Obtenha o histórico de trades para o símbolo
        trades = client.get_my_trades(symbol=symbol, limit=5)
        if not trades:
            return False, pd.DataFrame()
        
        # Ordenar trades por timestamp
        trades_sorted = sorted(trades, key=lambda x: x['time'], reverse=True)
        
        # Verificar se a última transação foi uma compra ou venda
        last_trade = trades_sorted[0]
        is_buy = last_trade['isBuyer']
        
        # Carregar o histórico de trades
        trade_history = read_trade_history(symbol)
        
        return is_buy, trade_history
    except exceptions.BinanceAPIException as e:
        logger.error(f"Erro na API Binance: {e}")
        time.sleep(25)
        return check_last_transaction(symbol)
    except Exception as e:
        logger.error(f"Erro inesperado ao verificar a última transação: {e}")
        time.sleep(25)
        return check_last_transaction(symbol)

def main_loop(symbol, quantity):
    global total_trade_duration, total_trades
    is_comprado_logged = False
    is_not_comprado_logged = False
    position_maintained = False

    while True:
        try:
            # Verifica o estado do bot
            is_buy, trade_history = check_last_transaction(symbol)

            if is_buy and not is_comprado_logged:
                logger.info(f"Bot v2 iniciado - Loop de venda para {symbol}.")
                is_comprado_logged = True
                is_not_comprado_logged = False

            if not is_buy and not is_not_comprado_logged:
                logger.info(f"Bot v2 iniciado - Loop de compra para {symbol}.")
                is_not_comprado_logged = True

            while is_buy:
                if not position_maintained:
                    logger.info(f"Loop de venda - Checando condições de venda para {symbol}.")
                    position_maintained = True
                ticker = float(client.get_symbol_ticker(symbol=symbol)['price'])

                if not trade_history.empty:
                    stoploss = trade_history['stoploss'].iloc[-1]
                    stopgain = trade_history['stopgain'].iloc[-1]

                    if ticker <= stoploss or ticker >= stopgain:
                        start_time = time.time()
                        balance = get_current_balance(symbol.split('USDT')[0])
                        lot_size = get_lot_size(symbol)
                        if balance > 0 and lot_size:
                            # Ajustar o saldo para atender ao tamanho do lote
                            quantity_to_sell = (balance // lot_size) * lot_size
                            if quantity_to_sell > 0:
                                order = client.order_market_sell(symbol=symbol, quantity=quantity_to_sell)
                                trade_duration = time.time() - start_time
                                sell_duration_metric.labels(symbol).observe(trade_duration)
                                total_trade_duration += trade_duration
                                is_buy = False
                                logger.info(f"Venda realizada para {symbol}.")
                                update_trade_history(trade_history, ticker, symbol)
                                update_metrics_on_sell(ticker, symbol)
                                position_maintained = False
                            else:
                                logger.info(f"Quantidade ajustada para venda é menor que o tamanho do lote para {symbol}.")
                                is_buy = False
                                position_maintained = False
                        else:
                            logger.info(f"Saldo insuficiente para venda para {symbol}.")
                            is_buy = False
                            position_maintained = False
                    time.sleep(1)

            while not is_buy:
                if not position_maintained:
                    logger.info(f"Loop de compra - Checando condições de compra para {symbol}.")
                    position_maintained = True
                klines = client.get_klines(symbol=symbol, interval=interval, limit=50)
                data = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

                # Convertendo para float e verificando se há falhas
                data['close'] = data['close'].apply(safe_float_conversion)
                data['low'] = data['low'].apply(safe_float_conversion)
                data['high'] = data['high'].apply(safe_float_conversion)
                data['volume'] = data['volume'].apply(safe_float_conversion)

                if data[['close', 'low', 'high', 'volume']].isnull().any().any():
                    logger.error(f"Dados corrompidos recebidos da API Binance para {symbol}.")
                    continue

                previous_ema = data['close'].ewm(span=9, adjust=False).mean().iloc[-2]
                pre_previous_ema = data['close'].ewm(span=9, adjust=False).mean().iloc[-3]
                current_price = data['close'].iloc[-1]
                previous_high = data['high'].iloc[-2]

                # Atualizar métricas constantemente
                current_price_metric.labels(symbol).set(current_price)
                current_high_price_metric.labels(symbol).set(data['high'].max())
                current_low_price_metric.labels(symbol).set(data['low'].min())
                current_volume_metric.labels(symbol).set(data['volume'].sum())
                price_standard_deviation_metric.labels(symbol).set(calculate_standard_deviation(data['close']))

                if previous_ema > pre_previous_ema and current_price >= previous_high:
                    start_time = time.time()
                    order = client.order_market_buy(symbol=symbol, quantity=quantity)
                    buy_duration_metric.labels(symbol).observe(time.time() - start_time)
                    stoploss = data['low'].iloc[-2]
                    stopgain = previous_high * 1.02
                    potential_loss = calculate_percentage(current_price, stoploss)
                    potential_gain = calculate_percentage(current_price, stopgain)
                    logger.info(f"Compramos {symbol} - Potencial de perda: {potential_loss:.2f}%, Potencial de ganho: {potential_gain:.2f}%")
                    is_buy = True
                    new_row = pd.DataFrame({
                        'horario': [datetime.now()],
                        'moeda': [symbol],
                        'valor_compra': [current_price],
                        'valor_venda': [None],
                        'quantidade_moeda': [quantity],
                        'max_referencia': [previous_high],
                        'min_referencia': [data['low'].iloc[-2]],
                        'stoploss': [stoploss],
                        'stopgain': [stopgain],
                        'potential_loss': [potential_loss],
                        'potential_gain': [potential_gain],
                        'timeframe': [interval],
                        'setup': [setup],
                        'outcome': [None]
                    })
                    trade_history = pd.concat([trade_history, new_row], ignore_index=True)
                    file_path = f'data/trade_history_{symbol}.csv'
                    trade_history.to_csv(file_path, index=False)
                    buy_prices.append(current_price)
                    update_metrics_on_buy(current_price, stoploss, stopgain, potential_loss, potential_gain, symbol)
                    is_not_comprado_logged = False
                    is_comprado_logged = False
                time.sleep(1)

        except exceptions.BinanceAPIException as e:
            logger.error(f"Erro na API Binance para {symbol}: {e}")
            time.sleep(25)  # Espera antes de tentar novamente
        except exceptions.BinanceOrderException as e:
            logger.error(f"Erro ao criar ordem na Binance para {symbol}: {e}")
            time.sleep(25)  # Espera antes de tentar novamente
        except (ConnectionError, Timeout) as e:
            logger.error(f"Erro de conexão para {symbol}: {e}")
            time.sleep(25)  # Espera antes de tentar novamente
        except Exception as e:
            logger.error(f"Erro inesperado para {symbol}: {e}")
            time.sleep(25)  # Espera antes de tentar novamente

if __name__ == "__main__":
    import threading
    threading.Thread(target=main_loop, args=(symbol_btc, quantity_btc)).start()
    threading.Thread(target=main_loop, args=(symbol_eth, quantity_eth)).start()
